In [29]:
import numpy as np
from project_helpers import gen_paths, payoff, forward, bs_put, bs_call
from project_network import fitting
import tensorflow as tf
import matplotlib.pyplot as plt

In [30]:
# Compute the continuation value Q for the N paths at time t_{m-1}
def continuation_q(weights, stock, delta_t, normalizingC): # normalizingC is normalizing constant from fitting t+1
    
    normalized_stock = stock / normalizingC
    w_1 = weights[0]  # first layer weights
    b_1 = weights[1]  # first layer biases
    w_2 = weights[2]  # second layer weights
    b_2 = weights[3][0]  # second layer bias
    p = len(w_2)  # number of hidden nodes in the first hidden layer
    cont = []  # continuation vector

    for s_tm in normalized_stock:
        sum_cond_exp = b_2
        cond_exp = 0
        for i in range(p):
            w_i = w_1[0][i]
            b_i = b_1[i]
            omega_i = w_2[i][0]

            if w_i >= 0 and b_i >= 0:
                cond_exp = w_i * forward(s_tm, r, delta_t) + b_i
            elif w_i > 0 > b_i:
                cond_exp = w_i * bs_call(s_tm, -b_i / w_i, delta_t, r, sigma)
            elif w_i < 0 < b_i:
                cond_exp = - w_i * bs_put(s_tm, -b_i / w_i, delta_t, r, sigma)
            elif w_i <= 0 and b_i <= 0:
                cond_exp = 0

            sum_cond_exp += omega_i * cond_exp

        # Discount by the risk-free rate
        cont.append(sum_cond_exp * np.exp(- r * delta_t))

    return cont

In [31]:
# Compute the value of exercising the option today, for the N samples
def exercise_h(stock, strike, style):
    h = [payoff(s, strike, style) for s in stock]
    return h

In [32]:
def model(S0, K, mu, sigma, N, monitoring_dates, style, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), model_weights=None):
    """
    monitoring_dates: t0=0, ..., tM=T
    """
    # generate sample paths of asset S
    sample_pathsS = gen_paths(monitoring_dates, S0, mu, sigma, N)

    # evaluate maturity time option values
    T = len(monitoring_dates) - 1
    option = np.zeros(sample_pathsS.shape)
    for n in range(N):
        S_T = sample_pathsS[n, T]
        option[n, T] = payoff(S_T, K, style)

    # fit the model at T, store fitted weights to initialize next regression
    betas = []  # store model weights
    fitted_beta = fitting(sample_pathsS[:, T], option[:, T],
                          weights=model_weights, optimizer=optimizer)
    betas.append(fitted_beta)

    # Compute option value at T-1
    h = exercise_h(sample_pathsS[:, T - 1], K, style)                           # value of exercising now
    time_increments = np.diff(monitoring_dates)
    q = continuation_q(fitted_beta, sample_pathsS[:, T - 1],                    # continuation value
                       time_increments[T - 1], np.max(sample_pathsS[:, T]))
    option[:, T - 1] = [max(h[i], q[i]) for i in range(N)]                      # take maximum of both values

    # compute option values by backward regression
    for m in range(T - 1, 0, -1):

        # fit new weights, initalize with previous weights
        fitted_beta = fitting(sample_pathsS[:, m], option[:, m], weights=fitted_beta, optimizer=optimizer)

        # compute estimated option value one time step earlier
        h = exercise_h(sample_pathsS[:, m - 1], K, style)                       # value of exercising now
        q = continuation_q(fitted_beta, sample_pathsS[:, m - 1],                # continuation value
                           time_increments[m - 1], np.max(sample_pathsS[:, m]))
        option[:, m - 1] = [max(h[i], q[i]) for i in range(N)]                  # take maximum of both values

        # append the model weights to the beta list
        betas.append(fitted_beta)

    # note that the estimated intial option value is the same for all samples (they all start in S0)
    initial_option_value = option[:, 0][0]

    return betas, initial_option_value
    

In [33]:
# Simulation parameters initialization
# The asset prices follow a Geometric Brownian Motion
S = 40  # Initial price
mu = 0.06  # Drift
sigma = 0.2  # Volatility
K = 40  # Strike price
r = 0.06  # Risk-free rate
T = 1  # Maturity
M = 10  # Number of monitoring dates
N = 10000  # Number of sample paths
pf_style = 'put'  # Payoff type
monitoring_dates = np.linspace(0, T, M + 1) 

In [34]:
weights, v_0 = model(S, K, mu, sigma, N, monitoring_dates, pf_style, optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001))
print(v_0)

Restoring model weights from the end of the best epoch: 2771.
Epoch 2777: early stopping


Restoring model weights from the end of the best epoch: 705.
Epoch 711: early stopping


Restoring model weights from the end of the best epoch: 536.
Epoch 542: early stopping


Restoring model weights from the end of the best epoch: 662.
Epoch 668: early stopping


Restoring model weights from the end of the best epoch: 618.
Epoch 624: early stopping


Restoring model weights from the end of the best epoch: 751.
Epoch 757: early stopping


Restoring model weights from the end of the best epoch: 713.
Epoch 719: early stopping


Restoring model weights from the end of the best epoch: 749.
Epoch 755: early stopping


Restoring model weights from the end of the best epoch: 815.
Epoch 821: early stopping


Restoring model weights from the end of the best epoch: 103.
Epoch 109: early stopping
2.130206844593108


In [35]:
# perform a few runs that reuse each others weights
v_0s = []
parameters = weights[-1]
for i in range(5):
    parameters, v_0 = model(S, K, mu, sigma, N, monitoring_dates, pf_style, optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001), model_weights=parameters)
    parameters = parameters[-1] # should this be parameters[0]?
    v_0s.append(v_0)

Restoring model weights from the end of the best epoch: 2174.
Epoch 2180: early stopping


Restoring model weights from the end of the best epoch: 676.
Epoch 682: early stopping


KeyboardInterrupt: 